In [1]:
import pandas as pd
import os
import sqlite3
import ast
import json
import numpy as np

# 1. Categorize Data

All columns and values are described here: https://transparency.dsa.ec.europa.eu/page/api-documentation#statement-attributes

The data is very heavy. There are many long strings like "DECISION_PROVISION_PARTIAL_SUSPENSION". We assign a unique code to each string, store that in a lookup table, and replace the strings with the codes in the data.

We do not categorize dates and times.

In [ ]:
lookup_table_path= '../data/processed/category_codes_q4.csv'

In [83]:
category_codes = pd.DataFrame(columns=['category', 'code'])
category_codes.to_csv(lookup_table_path, index=False)

In [2]:
def flatten(arr):
    """given a list of lists and other elements, returns a list with all elements"""
    flattened_arr = []
    for element in arr:
        if isinstance(element, list):
            flattened_arr.extend(element)
        else:
            flattened_arr.append(element)
    return flattened_arr

In [ ]:
def preprocess_value(value):
    """detects if a string is a list, if yes: for lists with only 1 element, returns the element. For longer lists, returns the original input"""
    if isinstance(value, str) and value.startswith('['):
        value = json.loads(value)
        if isinstance(value, list) and len(value) == 1:
            return value[0]
        elif isinstance(value, list):
            return json.dumps(value)
    return value

In [4]:
def preprocess_and_categorize(column, lookup_table):
    """given a column of a dataframe, replaces values with codes"""
    column = column.apply(preprocess_value)
    # Convert lists to strings
    column = column.apply(lambda x: json.dumps(x) if isinstance(x, list) else x)
    # Use the global Categorical object to assign codes
    mapping_dict = lookup_table.set_index('category').to_dict()['code']
    column = column.apply(lambda x: mapping_dict.get(x, None) if pd.notna(x) else None)
    return column

In [5]:
def categorize_df(df, mapping_table, ignore_cols=[]):
    """given da dataframe, a lookup table and a list of columns that should not be categorized, replaces strings with categoricals and stores codes and values in a lookup table"""
    
    original_df = df.copy()
    df = df.drop(columns=ignore_cols)
    # find all unique values in the df cells, turn them into a pandas categorical
    all_values = pd.Series(df.values.flatten())
    all_values = [json.loads(value) if isinstance(value, str) and value.startswith("[") else value for value in all_values]
    all_values = flatten(all_values)
    categorical = pd.Categorical(all_values)

    # Update the lookup table with new categories
    new_values = set(categorical.categories) - set(mapping_table["category"].values)
    new_entries = pd.DataFrame({"category": list(new_values), "code": range(len(mapping_table), len(new_values) + len(mapping_table))})
    mapping_table = pd.concat([mapping_table, new_entries], ignore_index=True)

    # Apply the preprocessing and categorization to all columns
    for col in df.columns:
        df[col] = preprocess_and_categorize(df[col])

    # Add the ignore columns back to the dataframe
    df = pd.concat([df, original_df[ignore_cols]], axis=1)

    return df, mapping_table

# Load and clean files, categorize, save in sql database

In [9]:
col_drops = ["uuid","content_language","illegal_content_legal_ground", "content_type", "decision_visibility_other","territorial_scope", "decision_monetary_other", "account_type", "decision_facts", "decision_ground_reference_url", "illegal_content_explanation", "incompatible_content_ground", "incompatible_content_explanation", "incompatible_content_illegal", "content_type_other", "category_specification_other", "source_identity", "platform_uid"]
date_cols = ["content_date", "application_date", "created_at"]

## 2. Store all data in an sqlite database

For quicker analysis and lighter data, we create a database

In [ ]:
#path to the merged csv files
path = "../data/raw/merged_csv/"
files = [f for f in os.listdir(path) if f.endswith('.csv')]
#path for the sqlite database
db_path = '../data/processed/2024-q4_data.db'
conn = sqlite3.connect(db_path)
for file in files:
    try:
        df = pd.read_csv(path + file)
        print("processing file: ", file)
        df = df.drop(columns=[col for col in col_drops if col in df.columns]) # Drop columns that are not needed
        df = df.astype({col: 'str' for col in df.select_dtypes(include='object').columns}) # Convert remaining object columns to strings       
        df, category_codes_q4 = categorize_df(df, category_codes_q4, ignore_cols=date_cols) # categorize the dataframe
        df.to_sql('categorized_data', conn, if_exists='append', index=False) # store the dataframe in the database
        print("stored in db")
        os.remove(path + file) # delete the file
        print("deleted file")
    except Exception as e:
        print(f"Error with file: {file}")
        print("Exception: ", e)
        pass
conn.close()

# Save category mapping

In [89]:
#category_codes_q4.to_csv('../data/processed/category_codes_q4_final.csv', index=False)

## Add column with decision-speed in the database

# Add columns for reaction speed

In [90]:
# Connect to the SQLite database
conn = sqlite3.connect(db_path)

conn.execute('''
            ALTER TABLE categorized_data ADD COLUMN decision_speed_seconds REAL;
            ''')

conn.execute('''
            UPDATE categorized_data
            SET decision_speed_seconds = 
            (JULIANDAY(application_date) - JULIANDAY(content_date))*86400;
             ''')
conn.commit()
conn.close()

# Optional: Add lookup table in db

## Store the lookup table in the database, too

In [14]:
# Connect to the SQLite database (or create it)
conn = sqlite3.connect(db_path)


# Connect to the SQLite database (or create it)
conn.execute('''
    CREATE TABLE IF NOT EXISTS category_lookup (
        category_code TEXT,
        category_name TEXT
    )
''')
# Insert data into the category_lookup table
for category_code, category_name in global_category_mapping.items():
    conn.execute("INSERT INTO category_lookup (category_code, category_name) VALUES (?, ?)", (category_name, category_code))
conn.commit()